**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

software = '/home/tony/DockM8/software'
protein_file = '/home/tony/DockM8/wocondock_main/2o1x_A_apo_protoss.pdb'
ref_file = '/home/tony/DockM8/wocondock_main/2o1x_A_lig_protoss.sdf'
docking_library = '/home/tony/DockM8/wocondock_main/Selection_of_FCHGroup_LeadLike.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS', 'QVINAW', 'QVINA2']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'RTMScore', 'SCORCH', 'vinardo']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'pkasolver'
# ncpus = int(os.cpu_count()-2)
ncpus = 7
pocket = 'reference'
#Create a temporary folder for all further calculations
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

[16:09:54] Initializing Normalizer
/home/tony/.conda/envs/dockm8/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/home/tony/.conda/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/tony/.conda/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.

The working directory has been set to: /home/tony/DockM8/wocondock_main
The folder: /home/tony/DockM8/wocondock_main/temp already exists


In [2]:
# if os.path.isfile(protein_file.replace('.pdb', '_pocket.pdb')) == False:
if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 8)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')


[2023-Jun-05 16:09:57]: Extracting pocket from /home/tony/DockM8/wocondock_main/2o1x_A_apo_protoss.pdb using /home/tony/DockM8/wocondock_main/2o1x_A_lig_protoss.sdf as reference ligand

[2023-Jun-05 16:10:08]: Finished extracting pocket from /home/tony/DockM8/wocondock_main/2o1x_A_apo_protoss.pdb using /home/tony/DockM8/wocondock_main/2o1x_A_lig_protoss.sdf as reference ligand
{'center': [16.7, -2.69, 17.61], 'size': [8, 8, 8]}


In [ ]:
if os.path.isfile(w_dir+'/temp/final_library.sdf') == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)

In [3]:
docking(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus, pocket_definition)


Splitting SDF file final_library.sdf ...


Splitting files: 100%|██████████| 9/9 [00:00<00:00, 559.55it/s]

Split docking library into 9 files each containing 1 compounds

[2023-Jun-05 16:10:08]: Docking split files using QVINAW...



Docking with QVINAW:   0%|          | 0/9 [00:00<?, ?it/s]


[2023-Jun-05 16:10:10]: ERROR: Failed to combine QVINAW SDF file!############################################################################
# If you used Qvina-W in your work, please cite:                           #
#                                                                          #
# Nafisa M. Hassan, Amr A. Alhossary, Yuguang Mu, and Chee-Keong Kwoh,     #
# Protein-Ligand Blind Docking Using QuickVina-W With Inter-Process        #
# Spatio-Temporal Integration, Scientific Reports 7(1):2017,               #
# doi:10.1038/s41598-017-15571-7                                           #
#                                                                          #
# You are also encouraged to cite:                                         #
# QuickVina 2                                                              #
# Amr Alhossary, Stephanus Daniel Handoko, Yuguang Mu, and Chee-Keong Kwoh,#
# Fast, Accurate, and Reliable Molecular Docking with QuickVina 2,         #
# Bioinfor

...

***************************************************************************************************
done.
Refining results ... *
searching finished in 70.826 seconds
done.

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1         -8.0      0.000      0.000
   2         -8.0      2.530      7.671
   3         -7.9      1.164      2.230
   4         -7.9      2.550      7.875
   5         -7.7      3.053      8.183
   6         -7.7      1.635      2.569
   7         -7.4      1.115      2.129
   8         -7.4      2.551      7.960
   9         -7.4      3.074      8.200
  10         -7.3      1.298      1.855
Writing output ... done.


Docking with QVINAW:  33%|███▎      | 3/9 [01:11<02:40, 26.72s/it]

********
done.
Refining results ... 
searching finished in 73.002 seconds
done.

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1         -8.3      0.000      0.000
   2         -7.9      2.569      3.820
   3         -7.8      3.322      4.457
   4         -7.8      3.650      8.893
   5         -7.5      3.158      8.842
   6         -7.3      3.364      4.387
   7         -7.2      3.756      5.226
   8         -7.1      3.060      8.846
   9         -7.0      5.717      7.558
  10         -6.8      5.134      8.031
Writing output ... done.


Docking with QVINAW:  44%|████▍     | 4/9 [01:14<01:25, 17.09s/it]

**********************

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')


In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

In [ ]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, 20)


In [ ]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', ['gnina', 'RTMScore'])